In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 展示高清图
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

In [ ]:
# 制作数据集

# 设定下载参数
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(0.1307, 0.3081)
])

# 下载训练集与测试集
train_Data = datasets.MNIST(
    root = 'D:/Jupyter/dataset/mnist/', 
    train = True, 
    download = True, 
    transform = transform
)
test_Data = datasets.MNIST(
    root = 'D:/Jupyter/dataset/mnist/', 
    train = False, 
    download = True, 
    transform = transform
)

In [ ]:
# 批次加载器
train_loader = DataLoader(train_Data, shuffle=True, batch_size=256)
test_loader  = DataLoader(test_Data, shuffle=False, batch_size=256)

CNN 类继承自 nn.Module，这是 PyTorch 中所有神经网络模型的基类。
__init__ 方法是类的构造函数，用于初始化网络的各个层。nn.Sequential 是一个容器，按顺序包含多个网络层。

nn.Conv2d(1, 6, kernel_size=5, padding=2)：第一个卷积层，输入通道数为1（灰度图像），输出通道数为6，卷积核大小为5x5，填充为2。
nn.Tanh()：激活函数，使用双曲正切函数。
nn.AvgPool2d(kernel_size=2, stride=2)：第一个平均池化层，池化窗口大小为2x2，步幅为2。
nn.Conv2d(6, 16, kernel_size=5)：第二个卷积层，输入通道数为6，输出通道数为16，卷积核大小为5x5。
nn.Tanh()：激活函数。
nn.AvgPool2d(kernel_size=2, stride=2)：第二个平均池化层，池化窗口大小为2x2，步幅为2。
nn.Conv2d(16, 120, kernel_size=5)：第三个卷积层，输入通道数为16，输出通道数为120，卷积核大小为5x5。
nn.Tanh()：激活函数。
nn.Flatten()：将多维张量展平成一维。
nn.Linear(120, 84)：全连接层，输入大小为120，输出大小为84。
nn.Tanh()：激活函数。
nn.Linear(84, 10)：全连接层，输入大小为84，输出大小为10（假设这是一个10类分类问题）。

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5), nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5), nn.Tanh(),
            nn.Flatten(),
            nn.Linear(120, 84), nn.Tanh(),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        y = self.net(x)
        return y

In [ ]:
# 查看网络结构
X = torch.rand(size= (1, 1, 28, 28))
for layer in CNN().net:
    X = layer(X)
    print( layer.__class__.__name__, 'output shape: \t', X.shape )

In [ ]:
# 创建子类的实例，并搬到GPU上
model = CNN().to('cuda:0')

In [ ]:
# 损失函数的选择
loss_fn = nn.CrossEntropyLoss()    # 自带softmax激活函数

In [ ]:
# 优化算法的选择
learning_rate = 0.9    # 设置学习率
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr = learning_rate, 
)

In [ ]:
# 训练网络
epochs = 5
losses = []        # 记录损失函数变化的列表

for epoch in range(epochs):
    for (x, y) in train_loader:                  # 获取小批次的x与y
        x, y = x.to('cuda:0'), y.to('cuda:0')    # 把小批次搬到GPU上
        Pred = model(x)                          # 一次前向传播（小批量）
        loss = loss_fn(Pred, y)                  # 计算损失函数
        losses.append(loss.item())               # 记录损失函数的变化
        optimizer.zero_grad()                    # 清理上一轮滞留的梯度
        loss.backward()                          # 一次反向传播
        optimizer.step()                         # 优化内部参数

Fig = plt.figure()
plt.plot(range(len(losses)), losses)
plt.show()

In [ ]:
# 测试网络
correct = 0
total = 0

with torch.no_grad():                                   # 该局部关闭梯度计算功能
    for (x, y) in test_loader:                          # 获取小批次的x与y
        x, y = x.to('cuda:0'), y.to('cuda:0')           # 把小批次搬到GPU上
        Pred = model(x)                                 # 一次前向传播（小批量）
        _, predicted = torch.max(Pred.data, dim=1)
        correct += torch.sum( (predicted == y) )
        total += y.size(0) 
        
print(f'测试集精准度: {100*correct/total} %')